In [41]:
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Embedding, Conv1D, Flatten, MaxPooling1D, GRU
from keras.models import Sequential
from keras.optimizers import Adam

In [32]:
def build_model_cnn_gru(X,
                        y,
                        vocabulary_size,
                        max_length,
                        callbacks_list=[
                            ModelCheckpoint(
                                filepath='CNN_GRU_best_weights.hdf5',
                                monitor='val_acc',
                                verbose=1,
                                save_best_only=True,
                                mode='max'),
                            EarlyStopping(
                                monitor='val_acc', patience=3, mode='max')
                        ],
                        Embedding_size=200,
                        batch_size=32,
                        validation_split=0.04,
                        epochs=100):
    """
    Create the model for a Convolutional Neural Network with Gated Recurrent Unit
    INPUT:
        X : Multidimensional list - The traning features
        y : list                  - The traning results
        callbacks_list :          - The callback options for the model
        Embedding_size            - The size of the embedding
        batch_size                - The size of the batch in the neural network
        validation_split          - The validation_test split
        epochs                    - The number of epochs
    OUTPUT:
        Returns the model trained and the history of the training
    """
    print('Using  Convolutional Neural Network with Gated Recurrent Unit')

    model_gru_cnn = Sequential()
    model_gru_cnn.add(
        Embedding(vocabulary_size, Embedding_size, input_length=max_length))
    model_gru_cnn.add(
        Conv1D(64, kernel_size=3, padding='same', activation='relu'))
    model_gru_cnn.add(MaxPooling1D(pool_size=2))
    model_gru_cnn.add(Dropout(0.25))
    model_gru_cnn.add(GRU(128, return_sequences=True))
    model_gru_cnn.add(Dropout(0.3))
    model_gru_cnn.add(Flatten())
    model_gru_cnn.add(Dense(128, activation='relu'))
    model_gru_cnn.add(Dropout(0.5))
    model_gru_cnn.add(Dense(1, activation='sigmoid'))
    model_gru_cnn.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.001),
        metrics=['accuracy'])

    model_gru_cnn.summary()

    history_gru_cnn = model_gru_cnn.fit(
        X,
        y,
        validation_split=validation_split,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks_list)

    return model_gru_cnn, history_gru_cnn


Loading the data

In [17]:
train_set = pd.read_pickle("./data/tweets.pkl")
tweets = list(train_set['tweet'].values)

In [19]:
# Tokenizing tweets
max_length = 32
vocabulary_size = 100000
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(tweets)
sequences = tokenizer.texts_to_sequences(tweets)

In [49]:
X = pad_sequences(sequences, maxlen=max_length)
y = (train_set['label'].values)

In [50]:
type(train_set['label'].values)

numpy.ndarray

In [52]:
# model, history = build_model_cnn_gru(X, y, vocabulary_size, max_length,
#                         callbacks_list=[
#                             ModelCheckpoint(
#                                 filepath='CNN_GRU_best_weights.hdf5',
#                                 monitor='val_acc',
#                                 verbose=1,
#                                 save_best_only=True,
#                                 mode='max'),
#                             EarlyStopping(
#                                 monitor='val_acc', patience=3, mode='max')
#                         ], validation_split=0.3, epochs=6)

model, history = build_model_cnn_gru(X, y, vocabulary_size,\
                                     max_length, callbacks_list=None, validation_split=0.3, epochs=4)

Using  Convolutional Neural Network with Gated Recurrent Unit
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 32, 200)           20000000  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 32, 64)            38464     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 16, 64)            0         
_________________________________________________________________
gru_5 (GRU)                  (None, 16, 128)           74112     
_________________________________________________________________
dropout_14 (Dropout)         (None, 16, 128)           0         
__________________________________________________________

c:\programdata\anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 126924 samples, validate on 54397 samples
Epoch 1/4
126924/126924 [==============================] - 1582s 12ms/step - loss: 0.3863 - accuracy: 0.8327 - val_loss: 0.8211 - val_accuracy: 0.5539
Epoch 2/4
126924/126924 [==============================] - 1596s 13ms/step - loss: 0.2915 - accuracy: 0.8786 - val_loss: 0.7844 - val_accuracy: 0.6273
Epoch 3/4
126924/126924 [==============================] - 1596s 13ms/step - loss: 0.2236 - accuracy: 0.9094 - val_loss: 0.8275 - val_accuracy: 0.6626
Epoch 4/4
126924/126924 [==============================] - 1597s 13ms/step - loss: 0.1718 - accuracy: 0.9311 - val_loss: 1.0539 - val_accuracy: 0.6167


In [53]:
model, history = build_model_cnn_gru(X, y, vocabulary_size, max_length,
                        callbacks_list=[
                            ModelCheckpoint(
                                filepath='CNN_GRU_best_weights.hdf5',
                                monitor='val_acc',
                                verbose=1,
                                save_best_only=True,
                                mode='max'),
                            EarlyStopping(
                                monitor='val_acc', patience=3, mode='max')
                        ], validation_split=0.3, epochs=6)

# model, history = build_model_cnn_gru(X, y, vocabulary_size,\
#                                      max_length, callbacks_list=None, validation_split=0.3, epochs=4)

Using  Convolutional Neural Network with Gated Recurrent Unit
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 32, 200)           20000000  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 32, 64)            38464     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 16, 64)            0         
_________________________________________________________________
gru_6 (GRU)                  (None, 16, 128)           74112     
_________________________________________________________________
dropout_17 (Dropout)         (None, 16, 128)           0         
__________________________________________________________

c:\programdata\anaconda3\envs\tensorflow_cpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 126924 samples, validate on 54397 samples
Epoch 1/6
126924/126924 [==============================] - 1587s 12ms/step - loss: 0.3870 - accuracy: 0.8328 - val_loss: 0.7666 - val_accuracy: 0.5777
Epoch 2/6


c:\programdata\anaconda3\envs\tensorflow_cpu\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
c:\programdata\anaconda3\envs\tensorflow_cpu\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


126924/126924 [==============================] - 1640s 13ms/step - loss: 0.2929 - accuracy: 0.8776 - val_loss: 0.7618 - val_accuracy: 0.5912
Epoch 3/6
 73440/126924 [================>.............] - ETA: 12:22 - loss: 0.2176 - accuracy: 0.9117

KeyboardInterrupt: 